In [1]:
import os
os.chdir('..')

from util import hand_to_str
from deck52 import random_deal
import binary

from nn.bid_info import BidInfo

binfo = BidInfo('../models/21gf_info/binfo_21gf-500000')


INFO:tensorflow:Restoring parameters from ../models/21gf_info/binfo_21gf-500000


In [2]:
auction = ['4C']
n_steps = 1 + (len(auction)-1) // 4
ns = -1
ew = -1
nesw_i = 1
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT85.AKT.K63.K8")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, ns, ew)


In [3]:
p_hcp, p_shp = binfo.model(A)
p_hcp = p_hcp.reshape((-1, n_steps, 3))[:, -1, :]
p_shp = p_shp.reshape((-1, n_steps, 12))[:, -1, :]

def f_trans_hcp(x): return 4 * x + 10
def f_trans_shp(x): return 1.75 * x + 3.25

p_hcp = f_trans_hcp(p_hcp)
p_shp = f_trans_shp(p_shp)

c_hcp = p_hcp.copy()
c_shp = p_shp.copy()

print(f"c_hcp:{c_hcp}")
print(f"c_shp:{c_shp}")


c_hcp:[[6.161647 6.62594  9.480374]]
c_shp:[[3.4721162 2.9833052 3.8504107 2.675741  3.4730241 3.2105064 3.6407146
  2.5272415 1.2402604 4.425272  1.6998398 5.7104445]]


In [4]:
auction = ['PAD_START', '2C','4N']
n_steps = 1 + len(auction) // 4
n_samples = 10
ns = -1
ew = -1
nesw_i = 3
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT8653.J.962.92")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, ns, ew)
p_hcp, p_shp = binfo.model(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[ 6.9078207 18.210526   9.56655  ]]


In [5]:
auction = ['PAD_START', '2C','4N','PASS']
n_steps = 1 + len(auction) // 4
n_samples = 10
ns = -1
ew = -1
nesw_i = 0
vuln = [True, False]
hand = binary.parse_hand_f(32)("K4.T9876532.54.6")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, ns, ew)
p_hcp, p_shp = binfo.model(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[20.275938   8.96347    6.9984803]
 [21.049858   8.790658   6.693444 ]]
